In [1]:
script = False

import os
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

if(script):
    from tqdm import tqdm
else: 
    from tqdm.notebook import tqdm

In [2]:
def make_one_plot(objects, title, data, Q, stretch, minimum):
    plt.figure(figsize=(4,4))
    rgb = make_lupton_rgb(objects[2], objects[1], objects[0], Q=Q, stretch=stretch, minimum=minimum)
    rgb = np.flipud(rgb)
    
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()
    
def make_one_plot_brit(objects, title, data):
    plt.figure(figsize=(4,4))
    
    imager = 0.0005*objects[2] 
    imageg = 3*objects[1] 
    imageb = 15*objects[0] 
    rgb = np.array([imager, imageg, imageb])
    #Making sure values are lower or equal to 255
    rgb = np.clip(rgb, 0, 255).astype(np.uint8)
    rgb = rgb.transpose(1, 2, 0)
    #Increasing contrast
    min_img, max_img = np.min(rgb), np.max(rgb)     
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min_img:max_img+1]=np.linspace(start=0,stop=255,num=(max_img-min_img)+1,endpoint=True,dtype=np.uint8)
    rgb = LUT[rgb]
    rgb = np.flipud(rgb)
                
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()

In [3]:
N = 500
filepath = 'data/in_sled2.fits'
hdu_list = fits.open(filepath)
images = hdu_list[1].data#[0:N]
data = pd.DataFrame(hdu_list[2].data)#.iloc[0:N]
hdu_list.close()

print(len(images), len(data))

1672 1672


In [4]:
data['#subject_id'] = range(0, len(data))
data['#NEW_ID'] = range(0, len(data))
data['#image_name_1'] = data['#NEW_ID'].apply(lambda x: f'pngs/sled_{x}_1.png')
data['#image_name_2'] = data['#NEW_ID'].apply(lambda x: f'pngs/sled_{x}_2.png')
data['#image_name_3'] = data['#NEW_ID'].apply(lambda x: f'pngs/sled_{x}_3.png')
data['#image_name_4'] = data['#NEW_ID'].apply(lambda x: f'pngs/sled_{x}_4.png')
data['!Subject Type'] = ['Test Subject']*len(data)
#data['#Type'] = ['SUB']*len(data)
#data['#SubType'] = ['SUB']*len(data)
#data['#Provenance'] = ['DES-VT-JG']*len(data)
data['Colours'] = ['gri']*len(data)
data['Size-in-arcsec'] = ['20']*len(data)
data['#CLASS'] = ['single_cand']*len(data)
#data['#REPORTED'] = [False]*len(data)
#data['#REF'] = [None]*len(data)
#data['!Previously published lens candidate?'] = ['No']*len(data)

In [5]:
col_drop = ['COADD_OBJECT_ID', 'K_RA', 'K_DEC', 'K_SCORE', 'Y3_COADD_OBJECT_ID',
       'J_RA', 'J_DEC', 'score_sims', 'score_real', 'score_both', 'G_RA',
       'G_DEC', 'SINGLE', 'RING', 'SMOOTH', 'COMPANIONS', 'SDSS_SPIRALS',
       'DES_SPIRALS', 'CROWDED', 'ARTIFACTS', 'MOST_NEGATIVES', 'index', 'ra',
       'dec', 'name', 'score', 'image_sep', 'info', 'n_img', 'flag',
       'image_conf', 'lens_type', 'source_type', 'contaminant_type', 'papers']

new_data = data.drop(col_drop, axis=1)

path = 'data/For_zooniverse/'
data.to_csv(path + 'complete_manifest.csv', index=False)
new_data.to_csv(path + 'manifest.csv', index=False)

print(new_data.columns)
print(data.columns)

Index(['#subject_id', '#NEW_ID', '#image_name_1', '#image_name_2',
       '#image_name_3', '#image_name_4', '!Subject Type', 'Colours',
       'Size-in-arcsec', '#CLASS'],
      dtype='object')
Index(['COADD_OBJECT_ID', 'K_RA', 'K_DEC', 'K_SCORE', 'Y3_COADD_OBJECT_ID',
       'J_RA', 'J_DEC', 'score_sims', 'score_real', 'score_both', 'G_RA',
       'G_DEC', 'SINGLE', 'RING', 'SMOOTH', 'COMPANIONS', 'SDSS_SPIRALS',
       'DES_SPIRALS', 'CROWDED', 'ARTIFACTS', 'MOST_NEGATIVES', 'index', 'ra',
       'dec', 'name', 'score', 'image_sep', 'info', 'n_img', 'flag',
       'image_conf', 'lens_type', 'source_type', 'contaminant_type', 'papers',
       '#subject_id', '#NEW_ID', '#image_name_1', '#image_name_2',
       '#image_name_3', '#image_name_4', '!Subject Type', 'Colours',
       'Size-in-arcsec', '#CLASS'],
      dtype='object')


In [6]:
png_path = path 

for i in tqdm(range(len(data))):
    #if(i == 5): break
    path_exists = png_path + data['#image_name_4'].iloc[i]
    if(os.path.exists(path_exists)):
        continue
    make_one_plot(images[i], data['#image_name_1'].iloc[i], data.iloc[i], 9, 60, (0,0,0)) # new first filter
    make_one_plot(images[i], data['#image_name_2'].iloc[i], data.iloc[i], 8, 40, (14,2,0)) # new second filter
    make_one_plot(images[i], data['#image_name_3'].iloc[i], data.iloc[i], 10, 37, (12,6,0)) # new third filter
    make_one_plot_brit(images[i], data['#image_name_4'].iloc[i], data.iloc[i])

  0%|          | 0/1672 [00:00<?, ?it/s]